Grupo 7
- André Silva
- Gabriel Sari
- José Cabadas
- Mirray Teixeira

In [50]:
import os
import pymongo
from dotenv import load_dotenv
import datetime

# Load environment variables from .env file
load_dotenv()

# Get MongoDB Atlas credentials from environment variables
mongo_username = os.getenv("MONGODB_ATLAS_USERNAME")
mongo_password = os.getenv("MONGODB_ATLAS_PASSWORD")
mongo_cluster_url = os.getenv("MONGODB_ATLAS_URL")

# Create a MongoClient and connect to the MongoDB Atlas cluster
db = pymongo.MongoClient(
    "mongodb+srv://" + mongo_username + ":" + mongo_password + "@" + mongo_cluster_url + "/unicamp?retryWrites=true&w=majority"
)

## Exercício 1

- Adicione alguns documentos em uma coleção chamada diaries utilizando os comandos abaixo 
- remova a coleção antes de começar e certifique-se de que ela esteja vazia.
```javascript
db.diaries.drop()
db.diaries.insertMany([
    { name: "dug", day: ISODate("2022-08-04"), txt: "went for a walk"},
    { name: "dug", day: ISODate("2022-08-06"), txt: "saw a squirrel"},
    { name: "ray", day: ISODate("2022-08-06"), txt: "met dug in the park"},
    { name: "dug", day: ISODate("2022-08-09"), txt: "got a treat"}
])
```

In [51]:
db = client.unicamp

db.diaries.drop()

results = db.diaries.insert_many([
    { "name": "dug", "day": datetime.datetime(2022, 8, 4), "txt": "went for a walk"},
    { "name": "dug", "day": datetime.datetime(2022, 8, 6), "txt": "saw a squirrel"},
    { "name": "ray", "day": datetime.datetime(2022, 8, 6), "txt": "met dug in the park"},
    { "name": "dug", "day": datetime.datetime(2022, 8, 9), "txt": "got a treat"}
])

for result in results.inserted_ids:
    print(f'Inserted _id: {result}')

Inserted _id: 640ce850bbc3864eb5fbc06d
Inserted _id: 640ce850bbc3864eb5fbc06e
Inserted _id: 640ce850bbc3864eb5fbc06f
Inserted _id: 640ce850bbc3864eb5fbc070


- Escreva um operação find() para retornar somente as entradas feitas pelo “dug”

In [74]:
dug_entries = db.diaries.find({'name': "dug"})
for entry in dug_entries:
    print(entry)

{'_id': ObjectId('640ce850bbc3864eb5fbc06d'), 'name': 'dug', 'day': datetime.datetime(2022, 8, 4, 0, 0), 'txt': 'went for a walk'}
{'_id': ObjectId('640ce850bbc3864eb5fbc06e'), 'name': 'dug', 'day': datetime.datetime(2022, 8, 6, 0, 0), 'txt': 'saw a squirrel'}
{'_id': ObjectId('640ce850bbc3864eb5fbc070'), 'name': 'dug', 'day': datetime.datetime(2022, 8, 9, 0, 0), 'txt': 'got a treat'}



- Agora modifique este comando para exibir apenas o registro da linha abaixo utilizando skip, limit e projection para isso.
```json
{ name: "dug", txt: "saw a squirrel" }

In [79]:

dug_entries = db.diaries.find({'name': "dug"}, {'_id': 0, 'name': 1, 'txt': 1}).skip(1).limit(1)

for entry in dug_entries:
    print(entry)

{'name': 'dug', 'txt': 'saw a squirrel'}


## Exercício 2
Utilizando a base de dados sample_training:
- Na coleção grades:
    - Contar os registros onde student_id seja <= 65
        - o total é de 660 documentos
        - você pode utilizar o método countDocuments({}) para isso

In [54]:
grades_collection = client.sample_training.grades

grades_collection.count_documents({'student_id': {'$lte': 65.0}})

660

- Na coleção inspections:
    - Contar registros com result igual a "Pass" o "Fail"
        - o total de documentos é 16609
        - pode ser escrito de duas formas diferentes

In [57]:
inspections_collection = client.sample_training.inspections

inspections_collection.count_documents({'result': {'$in': ["Pass", "Fail"]}})

16609

## Exercício 3
Utilizando a coleção companies:
- Qual a maior empresa até 200 funcinonários
- utilize o campo number_of_employees

In [68]:
companies_collection = client.sample_training.companies

company = companies_collection.find({'number_of_employees': {'$lte': 200}}).sort('number_of_employees', -1).limit(1)[0]

print(f'Company: {company["name"]}, number of employees: {company["number_of_employees"]}')

Company: Meebo, number of employees: 200


## Exercício 4
Utilizando a coleção inspections novamente execute os seguintes updades:
### Update 1
- todos os documentos com result: "Completed", alterar para result: "No Violation Issue"
- deve alterar 20 documentos

In [69]:
inspections_collection.update_many({'result': 'Completed'}, {'$set': {'result': 'No Violation Issue'}}).modified_count

20

### Update 2
- para os documentos com result: "Fail", criar un campo chamado "fine_value" com valor 100
- deve alterar 1100 documentos

In [70]:
inspections_collection.update_many({'result': 'Fail'}, {'$set': {'fine_value': 100}}).modified_count

1100

### Update 3
- para os documentos com "address.city": "ROSEDALE" e result: "Fail", incrementar "fine_value" em 150
- utilize o operador $inc
- deve alterar somente 1 documento

In [71]:
inspections_collection.update_one({'address.city': 'ROSEDALE', 'result': 'Fail'}, {'$inc': {'fine_value': 150}}).modified_count

1